In [ ]:
import json
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [ ]:
file_path = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/raw_prices_carlsjr_non_ca_05162024.csv"
ca_ff = pd.read_csv(file_path)
file_path_2 = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/uszips.csv"
ca_zip_count = pd.read_csv(file_path_2)
file_path_3 = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/processed_prices_ubereats_ca_ff_03222024.csv"
example = pd.read_csv(file_path_3)

In [ ]:
ca_ff

In [ ]:
ca_zip_count = ca_zip_count[['zip', 'county_name']]

In [ ]:
def price_list(x):
    return list(x)
def mean_non_zero(x):
    return np.mean(x[x != 0]) if np.any(x != 0) else 0

def median_non_zero(x):
    return np.median(x[x != 0]) if np.any(x != 0) else 0

def std_non_zero(x):
    return np.std(x[x != 0]) if np.any(x != 0) else 0

In [ ]:
#Drop all the columns we don't need 
ca_ff_ = ca_ff.drop(columns=['menu_item_calories'])

#Drop all restaurants with no restaurant address 
#ca_ff_ = ca_ff_.dropna(subset=['restaurant_location'])

#Run if doing restaurant-specific processing i.e. Wendy's Hardee's etc 

ca_ff_ = ca_ff_.rename(columns = {'restaurant_address': "restaurant_location"})
ca_ff_['restaurant_name'] = 'Carls Jr'
ca_ff_['restaurant_rating'] = np.nan
ca_ff_['number_of_ratings'] = np.nan

In [ ]:
# Define a function to split prices by "-" or "/"
def split_and_average(price_str):
    # Split the price string by "-" or "/"
    prices = re.split(r'[-/]', price_str)
    # Convert prices to float and calculate the average
    prices = [float(price) for price in prices]
    return sum(prices) / len(prices)

# Apply the split_and_average function to each row in the menu_item_price column
ca_ff_['menu_item_price'] = ca_ff_['menu_item_price'].apply(split_and_average)

In [ ]:
#cleaning up string columns 

ca_ff_['menu_item'] = ca_ff_['menu_item'].str.lower()
ca_ff_['restaurant_location'] = ca_ff_['restaurant_location'].str.lower()
ca_ff_['restaurant_name'] = ca_ff_['restaurant_name'].str.replace('_', ' ')

#remove special characters
ca_ff_['menu_item'] = ca_ff_['menu_item'].apply(lambda x: ''.join(ch for ch in x if ch.isalnum() or ch.isspace()))
ca_ff_

In [ ]:
#Filter to just Wendy's
ca_ff_wendy = ca_ff_[ca_ff_['restaurant_name'] == 'Wendy']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_wendy = ca_ff_wendy.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_wendy.columns = [' '.join(col).strip() for col in grouped_wendy.columns.values]

# #Second part of grouping 
wendy_lst = ['daves single', 'french fries', 'jr cheeseburger', 'jr hamburger']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_wendy = ca_ff_wendy[ca_ff_wendy['menu_item'].isin(wendy_lst)].sort_values('menu_item')
menu_items_wendy = menu_items_wendy.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_wendy_2 = menu_items_wendy.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_wendy_2[['specialty_item', 'fries', 'cheeseburger', 'hamburger']] = grouped_wendy_2['menu_item_price'].apply(pd.Series)
grouped_wendy_2.drop(columns=['menu_item_price'], inplace=True)

merged_wendy = pd.merge(grouped_wendy, grouped_wendy_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_wendy['combo'] = np.nan
merged_wendy

In [ ]:
#Filter to just Burger King 
ca_ff_bk = ca_ff_[ca_ff_['restaurant_name'] == 'Burger King']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_bk = ca_ff_bk.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_bk.columns = [' '.join(col).strip() for col in grouped_bk.columns.values]

# #Second part of grouping 
#they don't have a plain hamburger FOR NOW using whopper jr 
bk_lst = ['cheeseburger', 'french fries', 'whopper', 'whopper jr']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_bk = ca_ff_bk[ca_ff_bk['menu_item'].isin(bk_lst)].sort_values('menu_item')
menu_items_bk = menu_items_bk.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_bk_2 = menu_items_bk.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_bk_2[['cheeseburger', 'fries', 'specialty_item', 'hamburger']] = grouped_bk_2['menu_item_price'].apply(pd.Series)
grouped_bk_2.drop(columns=['menu_item_price'], inplace=True)

merged_bk = pd.merge(grouped_bk, grouped_bk_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_bk['combo'] = np.nan
merged_bk

In [ ]:
#Filter to Carl's Jr 

ca_ff_carls = ca_ff_[ca_ff_['restaurant_name'] == 'Carls Jr']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_carls = ca_ff_carls.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_carls.columns = [' '.join(col).strip() for col in grouped_carls.columns.values]


#Second part of grouping 
carls_lst = ['california classic double cheeseburger', 'single big carl']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_carls = ca_ff_carls[ca_ff_carls['menu_item'].isin(carls_lst)].sort_values('menu_item')
menu_items_carls = menu_items_carls.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_carls_2 = menu_items_carls.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_carls_2[['cheeseburger', 'specialty_item']] = grouped_carls_2['menu_item_price'].apply(pd.Series)
grouped_carls_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_carls = pd.merge(grouped_carls, grouped_carls_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_carls['fries'] = np.nan
merged_carls['combo'] = np.nan
merged_carls['hamburger'] = np.nan

merged_carls

In [ ]:
menu_items_carls['menu_item'].unique()

In [ ]:
pattern = r",\s*([a-zA-Z]{2})\s*,?\s*(\d{5}(?:-\d{4})?)"

def extract_state_zip(address):
    match = re.search(pattern, address)
    if match:
        state, zip_code = match.groups()
        return state, zip_code
    else:
        return None, None

# Apply the function to extract state and zip code
merged_carls[['state', 'zip']] = merged_carls['restaurant_location'].apply(lambda x: pd.Series(extract_state_zip(x)))
merged_carls['zip'] = merged_carls['zip'].str.split('-').str[0].astype(int)

#Get county 
merged_carls = merged_bk.merge(ca_zip_count, on = 'zip')

In [ ]:
specific_date = datetime.strptime('05162024', '%m%d%Y')
# Assign the datetime object to the entire 'date' column
merged_carls['date'] = specific_date
merged_carls['uber_eats'] = 0
merged_carls['post_policy'] = 1
merged_carls['fast_food'] = 1 
merged_carls['local'] = 0 

In [ ]:
merged_carls.to_csv("processed_prices_carlsjr_nonca_05162024.csv")